# Sandbox
Main Objective: <em>Simple, working, command-line based iteration of GAN Image Generator</em>
<br/><br/>
<u>Inputs</u><br/>
Train images, both true and imposter
<br/><br/>
<u>Outputs</u><br/>
Generated image mimicking true

## Phase I: Load Train Images

## Phase II: Build GAN

## Phase III: Train GAN

## Phase IV: Generate Images